In [1]:
import pandas as pd
import numpy as np
from cryptography.hazmat.primitives import serialization
from datetime import datetime
from src.clients import KalshiHttpClient, KalshiWebSocketClient, Environment
from src.web_scraper import get_kalshi_px, calculate_fair

env = Environment.PROD # toggle environment here
KEYFILE = '../kalshi/bsun1220.txt'
KEYID = pd.read_pickle('../kalshi/key.txt')

try:
    with open(KEYFILE, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None  # Provide the password if your key is encrypted
        )
except FileNotFoundError:
    raise FileNotFoundError(f"Private key file not found at {KEYFILE}")
except Exception as e:
    raise Exception(f"Error loading private key: {str(e)}")

# Initialize the HTTP client
client = KalshiHttpClient(
    key_id=KEYID,
    private_key=private_key,
    environment=env
)

In [3]:
#Params
DAILY_HOUR_EXPIRE = 17 #expiry of daily contract
FAIR_THRESHOLD = 0.05 #only trade contracts between x and 1 - x fair value
MIN_EDGE_REQ = 0.03 #minimum edge required to fire trade
MIN_HOUR = 1 #minimum number of hours until stop trading

In [15]:
def get_markets(bitcoin = True):
    now = datetime.now()
    if now.hour >= DAILY_HOUR_EXPIRE:
        now = now + timedelta(days=1)
    month_dict = ['', 'JAN', 'FEB' 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
    year = now.year %100
    symbol = 'BTC' if bitcoin else 'ETH'
    event_ticker = f'KX{symbol}D-{year}{month_dict[now.month]}{now.day}17'
    info = client.get(f'/trade-api/v2/events/{event_ticker}')

    tickers = [x['ticker'] for x in info['markets']]

    ans = {}
    for ticker in tickers:
        orderbook_url = f'/trade-api/v2/markets/{ticker}/orderbook'
        strike = int(np.round(float(ticker.split('-T')[1]) + 0.01, 2))
        ans[strike] = client.get(orderbook_url)
    
    return ans

In [13]:
for coin in ['ETH', 'BTC']:
    date_click = 0 if datetime.now().hour <= DAILY_HOUR_EXPIRE else 1
    is_btc = coin == 'BTC'
    ans = get_markets(is_btc)
    curr_px = get_kalshi_px(is_btc)
    fairs = calculate_fair(ans.keys(), coin, DAILY_HOUR_EXPIRE,curr_px, date_click)

    eligible_fairs ={}
    for strike in fairs:
        if fairs[strike] > FAIR_THRESHOLD and fairs[strike] < 1 - FAIR_THRESHOLD:
            eligible_fairs[strike] = fairs[strike]

    ans = get_markets(is_btc)
    eligible_markets = {}
    for strike in ans:
        if strike in eligible_fairs:
            eligible_markets[strike] = ans[strike]['orderbook']

In [14]:
eligible_markets, eligible_fairs

({103750: {'yes': [[1, 28987],
    [3, 20000],
    [4, 10110],
    [6, 888],
    [7, 888],
    [11, 11],
    [49, 50],
    [57, 12500],
    [72, 8000],
    [73, 50],
    [90, 42],
    [95, 10000],
    [96, 200],
    [98, 4728]],
   'no': [[1, 91]]}},
 {103750: 0.75})